In [ ]:
def CalculateMap(BandPair, FilePath,
                 TimePairs=TimePairs,  
                 BinMag=BinMag, BinColor=BinColor,
                 HashTableDim=HashTableDim, 
                 Objects=Objects, PointsPerDay=PointsPerDay, Thrs=Thrs):
    
    Band1 = BandPair[0]
    Band2 = BandPair[1]   

    with open(FilePath, 'rb') as f:
        Interp_load = pickle.load(f)
        TimeRange_load = pickle.load(f)    
        
    TotalObjNo = len(Interp_load[Band1])
        
    if TotalObjNo <= Objects[-1]:
        Objects = Objects[Objects<TotalObjNo]
        
    HashTable = np.zeros(HashTableDim[1:], dtype=np.uint32)
    
    for kk, TimePair in enumerate(TimePairs):
        
        dT1 = TimePair[0]
        dT2 = TimePair[1]

        dMag = []
        Color = []

        dMagMin = []
        dMagMax = []
        ColorMin = []
        ColorMax = []

        for II in Objects:

            if Interp_load[Band1][II]==[] or Interp_load[Band2][II]==[]:
                continue

            #Decide the range and the length of XX.
            TimeRangeStart = max( TimeRange_load[Band1][II][0], TimeRange_load[Band2][II][0] - dT1/1440, TimeRange_load[Band1][II][0] - dT2/1440 )
            TimeRangeEnd = min( TimeRange_load[Band1][II][1], TimeRange_load[Band1][II][1] - dT2/1440, TimeRange_load[Band2][II][1] - dT1/1440 )

            TimeRange = TimeRangeEnd - TimeRangeStart
            if TimeRange<=0:
                continue
                
            SampleNo = int(PointsPerDay*TimeRange)

            XX = np.random.rand(SampleNo)*TimeRange + TimeRangeStart

            #Calculate the values of the functions with selected x values.  
            Mag1 = Interp_load[Band1][II](XX)
            Mag2 = Interp_load[Band2][II](XX+dT1/1440)
            Mag12 = Interp_load[Band1][II](XX+dT2/1440)

            #Add a threshold for the results and output.
            Mask = (Mag1<Thrs[Band1]) * (Mag2<Thrs[Band2]) * (Mag12<Thrs[Band1])

            dMag.extend(  (Mag1[Mask] - Mag12[Mask])*np.sign(dT2) )
            Color.extend(Mag1[Mask] - Mag2[Mask])

        # data = np.array([dMag, Color])
        histdata,_,_ = np.histogram2d(dMag, Color, bins=[BinMag, BinColor])

        dMagMin.append(min(dMag))
        dMagMax.append(max(dMag))
        ColorMin.append(min(Color))
        ColorMax.append(max(Color))

        outliersNo = len(dMag) - int(histdata.sum())

        HashTable[kk] = histdata
            
    return  [len(Objects), BandPair, outliersNo, min(dMagMin), max(dMagMax), min(ColorMin), max(ColorMax)], HashTable